<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/09_LogReg_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression on 100 cycles

The file contains 2 version of computation:

2.   Having the real data, consider a part as 1 and another as 0 of the same size
3.   Consider the real data as 1 and create unattested colexifications considered 0

In the case 2, the Regression model tries to predict the most probable colexifications; in the case 3 it tries to predict whether a pair is at all a colexification.

The file outputs two dataframes: df_logreg_bal.csv (balanced) and df_logreg_un.csv (unattested).

NB: it is used the pre-trained version of FastText.

In [1]:
# these lines print the time of execution in every cell
!pip install ipython-autotime
%load_ext autotime

time: 3.52 ms (started: 2021-05-07 10:50:14 +00:00)


In [2]:
pip install fasttext

     |████████████████████████████████| 71kB 4.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3096131 sha256=f48fb82b866a76a57cb218091d63f0761f6ba0c6787552f760843fc00aabc061
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext
time: 41.8 s (started: 2021-05-07 10:50:17 +00:00)


In [3]:
import fasttext.util
from google.colab import files
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import time
import random

time: 1.04 s (started: 2021-05-07 10:52:18 +00:00)


In [4]:
import itertools

def cos_sim(a, b):
	"""Takes 2 vectors a, b and returns the cosine similarity according 
	to the definition of the dot product
	"""
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)


# function to write in file
def log(w):
  with open('oov_log.txt', 'a+') as file:
    file.write(w)


# function to handle oov word exception
def trySimilarity(w1, w2, default, model):
  # calculate cosine similarity between two vectors if those exists in the model, else return default --> makes sense? ft deals with OOV
  try:
    v1 = model.get_sentence_vector(w1)
  except:
    log(w1)
    return default
  try:
    v2 = model.get_sentence_vector(w2)
  except:
    log(w2)
    return default
  return cos_sim(v1, v2)


def add_cosine_sim(df,col1,col2,ft_model):
  """
  Enriches dataframe with cosine similarity
  Input: a dataframe, two column names, and a fastText mdodel
  Output: The inputted dataframe with a new column with cosine similarities
  """

  #query the model and if there is an oov word write that word to oov_log.txt and move to the next word

  #add column with cosine sims row-wise

  df['Cosine'] = df.apply(lambda row: trySimilarity(row[col1], row[col2], 0, ft_model), axis=1) 

  return df

time: 15.9 ms (started: 2021-05-07 10:52:23 +00:00)


In [5]:
!gdown --id 1tu36NAL3MyHMkajE7wEDffbF0OI5fidY #listBoot
# bootstrappping with 100 as number of cicle

Downloading...
From: https://drive.google.com/uc?id=1tu36NAL3MyHMkajE7wEDffbF0OI5fidY
To: /content/listBoot.txt
222MB [00:01, 208MB/s]
time: 3.84 s (started: 2021-05-07 10:52:35 +00:00)


In [6]:
with open("listBoot.txt", "rb") as fp:   # Unpickling
  lBoot = pickle.load(fp)

time: 9.35 s (started: 2021-05-07 10:52:44 +00:00)


### Version 2

In [7]:
def timer(start,end):
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print('TIME: '+"{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  print('\n')

def calculate_logreg(nB):  
  
  #print ("Running calculate_logreg function")
  start_time = time.time()

  listLogReg = []


  for k in range (nB):
    nBoot = k

    #print ("Creating dataframe from bootstrap cicle")
    df = pd.DataFrame(lBoot[nBoot], columns=['Colex_pair','Concept1','Concept2', 'Prob']) # to save in df
    df = df.sort_values(by=['Prob'])
    df = df.reset_index(drop = True)

    # obtain cosine values from model
    #print("Obtaining cosine similarity from model")
    add_cosine_sim(df,'Concept1','Concept2', ft)

    # assign 1 to the 2% entries with highest probability
    totN = df.shape[0]
    selected = round(totN * 0.02)
    df ['Colex_bool'] = 0
    df.loc[( totN - selected) : ,'Colex_bool'] = 1

    # select the same number of 0 and 1 pairs --> balance
    g = df.groupby('Colex_bool')
    g = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True)))

    X = g[['Cosine']]
    y = g['Colex_bool']

    #print("Calculating logistic regression scores")
    for i in range(10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
      logreg = LogisticRegression()
      logreg.fit(X_train, y_train)
      y_pred = logreg.predict(X_test)
      f1 = round(f1_score(y_test,y_pred, average='macro'), 2)
      newEl = [nBoot, i, f1] # row: number of boot, random_state, f1 result
      listLogReg.append(newEl)

    print("Computation finished for boot number ", k, '\n')
    timer(start_time, time.time())

  return listLogReg  

time: 38.7 ms (started: 2021-05-07 10:52:56 +00:00)


### Version 3
Create unattested colexifications ("fake colexifications")

In [8]:
def calculate_logreg_un(nB):  
  
  #print ("Running calculate_logreg_un function")
  start_time = time.time()

  listLogReg_un = []


  for k in range (nB):
    nBoot = k

    #print ("Creating dataframe from bootstrap cicle")
    df = pd.DataFrame(lBoot[nBoot], columns=['Colex_pair','Concept1','Concept2', 'Prob']) # to save in df
    df['Colex_bool'] = 1 # all the existing colexifications are real

    real_col = set(list(df['Colex_pair'])) # remove duplicates

    # create list with single items
    real_col_l = [item for t in real_col for item in t]
    real_col_l = list(set(real_col_l)) # delete duplicates

    # create unattested colexifications
    #print ("Creating unattested colexifications")
    totN = df.shape[0]
    un_col = []
    for i in range (totN):
      while True:
        el1 = random.choice(real_col_l)
        el2 = random.choice(real_col_l)
        while el1 == el2:
          el2 = random.choice(real_col_l)
        supp_t = (el1, el2)
        if supp_t not in real_col: # fail condition
          un_col.append(supp_t) # append only tuple that is not in real ones
          break

    # create dataframe with 0 values: unattested colexifications
    df_un = pd.DataFrame(un_col, columns =['Concept1', 'Concept2'])
    df_un['Colex_pair'] = un_col
    df_un['Colex_bool'] = 0

    # join two dataframes
    #print ("Ultimated dataframe with 0 and 1 values")
    df = df.append(df_un)

    # obtain cosine values from model
    #print("Obtaining cosine similarity from model")
    add_cosine_sim(df,'Concept1','Concept2', ft)

    # Logistic Regression
    X = df[['Cosine']]
    y = df['Colex_bool']

    #print("Calculating logistic regression scores")
    for i in range(10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
      logreg = LogisticRegression()
      logreg.fit(X_train, y_train)
      y_pred = logreg.predict(X_test)
      f1 = round(f1_score(y_test,y_pred, average='macro'), 2)
      newEl = [nBoot, i, f1] # row: number of boot, random_state, f1 result
      listLogReg_un.append(newEl)

    print("Computation finished for boot number ", k, '\n')
    timer(start_time, time.time())

  return listLogReg_un

time: 38.5 ms (started: 2021-05-07 10:53:01 +00:00)


### Check if small model is comparable with big model (1 GB corpus vs 13 GB)

In [9]:
# downloading models
!gdown --id 1cGZcSpdX_eHb0eO_2-W73wyMjsIa1HwC # wiki_5
!gdown --id 1RQMk998Y7ZgNpBBS2UXOcEINLsa9iTvl # wiki1_5

Downloading...
From: https://drive.google.com/uc?id=1cGZcSpdX_eHb0eO_2-W73wyMjsIa1HwC
To: /content/wiki_5.bin
2.66GB [00:29, 91.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RQMk998Y7ZgNpBBS2UXOcEINLsa9iTvl
To: /content/wiki1_5.bin
1.08GB [00:16, 65.5MB/s]
time: 1min 45s (started: 2021-05-07 10:53:10 +00:00)


In [10]:
random.seed(70)
models = ["wiki_5.bin", "wiki1_5.bin"]
n_cycles = 20 # reduced number of cycles to speed up

for m in models:
  ft = fasttext.load_model(m) # load model
  print ("Working with model "+ m)
  values = calculate_logreg(n_cycles)
  df_logreg_bal = pd.DataFrame(values, columns=['boot_df', 'random_state', 'f1_score'])
  name_bal = "bal_" + m[:-4] + ".csv"
  df_logreg_bal.to_csv(name_bal, index = False)
  values_un = calculate_logreg_un(n_cycles)
  df_logreg_un = pd.DataFrame(values_un, columns=['boot_df', 'random_state', 'f1_score'])
  name_un = "un_" + m[:-4] + ".csv"
  df_logreg_un.to_csv(name_un, index = False)
  print ("\n")

Working with model wiki_5.bin
Computation finished for boot number  0 

TIME: 00:00:03.37


Computation finished for boot number  1 

TIME: 00:00:05.78


Computation finished for boot number  2 

TIME: 00:00:08.38


Computation finished for boot number  3 

TIME: 00:00:10.66


Computation finished for boot number  4 

TIME: 00:00:14.80


Computation finished for boot number  5 

TIME: 00:00:17.52


Computation finished for boot number  6 

TIME: 00:00:20.13


Computation finished for boot number  7 

TIME: 00:00:22.31


Computation finished for boot number  8 

TIME: 00:00:25.01


Computation finished for boot number  9 

TIME: 00:00:27.17


Computation finished for boot number  10 

TIME: 00:00:29.80


Computation finished for boot number  11 

TIME: 00:00:32.50


Computation finished for boot number  12 

TIME: 00:00:35.09


Computation finished for boot number  13 

TIME: 00:00:37.23


Computation finished for boot number  14 

TIME: 00:00:39.87


Computation finished for boot numbe

Working with model wiki1_5.bin
Computation finished for boot number  0 

TIME: 00:00:04.03


Computation finished for boot number  1 

TIME: 00:00:06.49


Computation finished for boot number  2 

TIME: 00:00:09.21


Computation finished for boot number  3 

TIME: 00:00:11.67


Computation finished for boot number  4 

TIME: 00:00:14.24


Computation finished for boot number  5 

TIME: 00:00:17.08


Computation finished for boot number  6 

TIME: 00:00:19.87


Computation finished for boot number  7 

TIME: 00:00:22.15


Computation finished for boot number  8 

TIME: 00:00:24.91


Computation finished for boot number  9 

TIME: 00:00:27.16


Computation finished for boot number  10 

TIME: 00:00:29.89


Computation finished for boot number  11 

TIME: 00:00:32.69


Computation finished for boot number  12 

TIME: 00:00:35.45


Computation finished for boot number  13 

TIME: 00:00:37.66


Computation finished for boot number  14 

TIME: 00:00:40.41


Computation finished for boot numb

In [13]:
df_bal_wiki1_5 = pd.read_csv('bal_wiki1_5.csv')
df_bal_wiki_5 = pd.read_csv('bal_wiki_5.csv')
df_un_wiki1_5 = pd.read_csv('un_wiki1_5.csv')
df_un_wiki_5 = pd.read_csv('un_wiki_5.csv')

time: 18.1 ms (started: 2021-05-07 11:04:53 +00:00)


In [16]:
print ("Average value for small model balanced "+ str (df_bal_wiki1_5['f1_score'].mean()))
print ("Average value for big balanced "+ str (df_bal_wiki_5['f1_score'].mean()))
print ("Average value for small unattested "+str (df_un_wiki1_5['f1_score'].mean()))
print ("Average value for big unattested "+ str (df_un_wiki_5['f1_score'].mean()))

Average value for small model balanced 0.7405999999999996
Average value for big balanced 0.7439999999999997
Average value for small unattested 0.5982999999999988
Average value for big unattested 0.600999999999999
time: 7.61 ms (started: 2021-05-07 11:07:14 +00:00)


We can say that the two models are comparable

### Pipeline

In [17]:
# downloading models
!gdown --id 1iWJpPmMKdEhj7PfsjteescA9Jd7UN1SD # wiki1_1
!gdown --id 1jiQHwwr3v76EgZwdWClnoE8me3bb4ivS # wiki1_2
!gdown --id 13OtvEg3xeE_hTFKcPt0QN8qcp4UO7n0Y # wiki1_3
# wiki1_5 is already downloaded
!gdown --id 1VXfD4Ve-zH0v-CjY6a9Q21xDKgxLLACx # wiki1_7
!gdown --id 1QrnfYKFAOS-pXqzeAnfDBFjvtcpmjhSr # wiki1_10
!gdown --id 1PAfysoCq6xYmAM_qxwl93CukTls9dLkH # wiki1_20
!gdown --id 1Oxz_wS6TYo95GMSFDGwkcugKxglEKdef # wiki1_50

Downloading...
From: https://drive.google.com/uc?id=1iWJpPmMKdEhj7PfsjteescA9Jd7UN1SD
To: /content/wiki1_1.bin
1.08GB [00:07, 146MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jiQHwwr3v76EgZwdWClnoE8me3bb4ivS
To: /content/wiki1_2.bin
1.08GB [00:16, 63.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=13OtvEg3xeE_hTFKcPt0QN8qcp4UO7n0Y
To: /content/wiki1_3.bin
1.08GB [00:14, 72.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VXfD4Ve-zH0v-CjY6a9Q21xDKgxLLACx
To: /content/wiki1_7.bin
1.08GB [00:17, 63.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QrnfYKFAOS-pXqzeAnfDBFjvtcpmjhSr
To: /content/wiki1_10.bin
1.08GB [00:15, 69.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PAfysoCq6xYmAM_qxwl93CukTls9dLkH
To: /content/wiki1_20.bin
1.08GB [00:17, 62.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oxz_wS6TYo95GMSFDGwkcugKxglEKdef
To: /content/wiki1_50.bin
1.08GB [00:14, 76.1MB/s]
time: 3min 56s (started: 2021-05-07 11:13:11 +

In [21]:
random.seed(70)
models = ["wiki1_1.bin", "wiki1_2.bin", "wiki1_3.bin", "wiki1_5.bin", "wiki1_7.bin", "wiki1_10.bin", "wiki1_20.bin", "wiki1_50.bin"]
n_cycles = len(lBoot) #100

for m in models:
  ft = fasttext.load_model(m) # load model
  print ("Working with model "+ m)
  print ("Balanced version")
  values = calculate_logreg(n_cycles)
  df_logreg_bal = pd.DataFrame(values, columns=['boot_df', 'random_state', 'f1_score'])
  name_bal = "df_bal_" + m[:-4] + ".csv"
  df_logreg_bal.to_csv(name_bal, index = False)
  print ("Unattested version")
  values_un = calculate_logreg_un(n_cycles)
  df_logreg_un = pd.DataFrame(values_un, columns=['boot_df', 'random_state', 'f1_score'])
  name_un = "df_un_" + m[:-4] + ".csv"
  df_logreg_un.to_csv(name_un, index = False)
  print ("\n")

Working with model wiki1_1.bin
Balanced version
Computation finished for boot number  0 

TIME: 00:00:02.79


Computation finished for boot number  1 

TIME: 00:00:05.37


Computation finished for boot number  2 

TIME: 00:00:08.17


Computation finished for boot number  3 

TIME: 00:00:10.65


Computation finished for boot number  4 

TIME: 00:00:13.35


Computation finished for boot number  5 

TIME: 00:00:16.25


Computation finished for boot number  6 

TIME: 00:00:19.09


Computation finished for boot number  7 

TIME: 00:00:21.47


Computation finished for boot number  8 

TIME: 00:00:24.31


Computation finished for boot number  9 

TIME: 00:00:26.60


Computation finished for boot number  10 

TIME: 00:00:29.38


Computation finished for boot number  11 

TIME: 00:00:32.28


Computation finished for boot number  12 

TIME: 00:00:35.03


Computation finished for boot number  13 

TIME: 00:00:37.30


Computation finished for boot number  14 

TIME: 00:00:40.13


Computation finis

Working with model wiki1_2.bin
Balanced version
Computation finished for boot number  0 

TIME: 00:00:02.71


Computation finished for boot number  1 

TIME: 00:00:05.28


Computation finished for boot number  2 

TIME: 00:00:08.12


Computation finished for boot number  3 

TIME: 00:00:10.62


Computation finished for boot number  4 

TIME: 00:00:13.39


Computation finished for boot number  5 

TIME: 00:00:16.33


Computation finished for boot number  6 

TIME: 00:00:19.28


Computation finished for boot number  7 

TIME: 00:00:21.63


Computation finished for boot number  8 

TIME: 00:00:24.54


Computation finished for boot number  9 

TIME: 00:00:26.86


Computation finished for boot number  10 

TIME: 00:00:29.70


Computation finished for boot number  11 

TIME: 00:00:32.59


Computation finished for boot number  12 

TIME: 00:00:35.44


Computation finished for boot number  13 

TIME: 00:00:37.75


Computation finished for boot number  14 

TIME: 00:00:40.62


Computation finis

Working with model wiki1_3.bin
Balanced version
Computation finished for boot number  0 

TIME: 00:00:02.76


Computation finished for boot number  1 

TIME: 00:00:06.70


Computation finished for boot number  2 

TIME: 00:00:09.54


Computation finished for boot number  3 

TIME: 00:00:12.04


Computation finished for boot number  4 

TIME: 00:00:14.72


Computation finished for boot number  5 

TIME: 00:00:17.65


Computation finished for boot number  6 

TIME: 00:00:20.52


Computation finished for boot number  7 

TIME: 00:00:22.86


Computation finished for boot number  8 

TIME: 00:00:25.75


Computation finished for boot number  9 

TIME: 00:00:28.11


Computation finished for boot number  10 

TIME: 00:00:30.91


Computation finished for boot number  11 

TIME: 00:00:33.74


Computation finished for boot number  12 

TIME: 00:00:36.51


Computation finished for boot number  13 

TIME: 00:00:38.82


Computation finished for boot number  14 

TIME: 00:00:41.71


Computation finis

Working with model wiki1_5.bin
Balanced version
Computation finished for boot number  0 

TIME: 00:00:02.80


Computation finished for boot number  1 

TIME: 00:00:05.36


Computation finished for boot number  2 

TIME: 00:00:08.23


Computation finished for boot number  3 

TIME: 00:00:10.75


Computation finished for boot number  4 

TIME: 00:00:13.46


Computation finished for boot number  5 

TIME: 00:00:16.40


Computation finished for boot number  6 

TIME: 00:00:19.30


Computation finished for boot number  7 

TIME: 00:00:21.66


Computation finished for boot number  8 

TIME: 00:00:24.60


Computation finished for boot number  9 

TIME: 00:00:28.37


Computation finished for boot number  10 

TIME: 00:00:31.22


Computation finished for boot number  11 

TIME: 00:00:34.12


Computation finished for boot number  12 

TIME: 00:00:36.92


Computation finished for boot number  13 

TIME: 00:00:39.20


Computation finished for boot number  14 

TIME: 00:00:42.09


Computation finis

Working with model wiki1_7.bin
Balanced version
Computation finished for boot number  0 

TIME: 00:00:02.79


Computation finished for boot number  1 

TIME: 00:00:05.29


Computation finished for boot number  2 

TIME: 00:00:08.14


Computation finished for boot number  3 

TIME: 00:00:10.70


Computation finished for boot number  4 

TIME: 00:00:13.41


Computation finished for boot number  5 

TIME: 00:00:16.26


Computation finished for boot number  6 

TIME: 00:00:19.16


Computation finished for boot number  7 

TIME: 00:00:21.47


Computation finished for boot number  8 

TIME: 00:00:24.35


Computation finished for boot number  9 

TIME: 00:00:26.62


Computation finished for boot number  10 

TIME: 00:00:29.41


Computation finished for boot number  11 

TIME: 00:00:32.24


Computation finished for boot number  12 

TIME: 00:00:35.08


Computation finished for boot number  13 

TIME: 00:00:38.87


Computation finished for boot number  14 

TIME: 00:00:44.85


Computation finis

Working with model wiki1_10.bin
Balanced version
Computation finished for boot number  0 

TIME: 00:00:02.76


Computation finished for boot number  1 

TIME: 00:00:05.27


Computation finished for boot number  2 

TIME: 00:00:08.08


Computation finished for boot number  3 

TIME: 00:00:10.58


Computation finished for boot number  4 

TIME: 00:00:13.38


Computation finished for boot number  5 

TIME: 00:00:16.51


Computation finished for boot number  6 

TIME: 00:00:19.52


Computation finished for boot number  7 

TIME: 00:00:21.99


Computation finished for boot number  8 

TIME: 00:00:24.99


Computation finished for boot number  9 

TIME: 00:00:27.37


Computation finished for boot number  10 

TIME: 00:00:30.29


Computation finished for boot number  11 

TIME: 00:00:33.31


Computation finished for boot number  12 

TIME: 00:00:36.23


Computation finished for boot number  13 

TIME: 00:00:38.61


Computation finished for boot number  14 

TIME: 00:00:41.59


Computation fini

Working with model wiki1_20.bin
Balanced version
Computation finished for boot number  0 

TIME: 00:00:04.37


Computation finished for boot number  1 

TIME: 00:00:08.19


Computation finished for boot number  2 

TIME: 00:00:11.83


Computation finished for boot number  3 

TIME: 00:00:14.39


Computation finished for boot number  4 

TIME: 00:00:17.31


Computation finished for boot number  5 

TIME: 00:00:20.28


Computation finished for boot number  6 

TIME: 00:00:24.69


Computation finished for boot number  7 

TIME: 00:00:27.07


Computation finished for boot number  8 

TIME: 00:00:29.91


Computation finished for boot number  9 

TIME: 00:00:32.31


Computation finished for boot number  10 

TIME: 00:00:35.13


Computation finished for boot number  11 

TIME: 00:00:37.99


Computation finished for boot number  12 

TIME: 00:00:40.73


Computation finished for boot number  13 

TIME: 00:00:42.95


Computation finished for boot number  14 

TIME: 00:00:45.81


Computation fini

Working with model wiki1_50.bin
Balanced version
Computation finished for boot number  0 

TIME: 00:00:02.76


Computation finished for boot number  1 

TIME: 00:00:05.33


Computation finished for boot number  2 

TIME: 00:00:08.15


Computation finished for boot number  3 

TIME: 00:00:10.60


Computation finished for boot number  4 

TIME: 00:00:13.34


Computation finished for boot number  5 

TIME: 00:00:16.20


Computation finished for boot number  6 

TIME: 00:00:19.04


Computation finished for boot number  7 

TIME: 00:00:21.46


Computation finished for boot number  8 

TIME: 00:00:24.37


Computation finished for boot number  9 

TIME: 00:00:26.79


Computation finished for boot number  10 

TIME: 00:00:29.61


Computation finished for boot number  11 

TIME: 00:00:32.55


Computation finished for boot number  12 

TIME: 00:00:35.36


Computation finished for boot number  13 

TIME: 00:00:37.69


Computation finished for boot number  14 

TIME: 00:00:40.54


Computation fini

### Checking results

In [22]:
df_bal_wiki1_1 = pd.read_csv('df_bal_wiki1_1.csv')
df_bal_wiki1_2 = pd.read_csv('df_bal_wiki1_2.csv')
df_bal_wiki1_3 = pd.read_csv('df_bal_wiki1_3.csv')
df_bal_wiki1_5 = pd.read_csv('df_bal_wiki1_5.csv')
df_bal_wiki1_7 = pd.read_csv('df_bal_wiki1_7.csv')
df_bal_wiki1_10 = pd.read_csv('df_bal_wiki1_10.csv')
df_bal_wiki1_20 = pd.read_csv('df_bal_wiki1_20.csv')
df_bal_wiki1_50 = pd.read_csv('df_bal_wiki1_50.csv')


df_un_wiki1_1 = pd.read_csv('df_un_wiki1_1.csv')
df_un_wiki1_2 = pd.read_csv('df_un_wiki1_2.csv')
df_un_wiki1_3 = pd.read_csv('df_un_wiki1_3.csv')
df_un_wiki1_5 = pd.read_csv('df_un_wiki1_5.csv')
df_un_wiki1_7 = pd.read_csv('df_un_wiki1_7.csv')
df_un_wiki1_10 = pd.read_csv('df_un_wiki1_10.csv')
df_un_wiki1_20 = pd.read_csv('df_un_wiki1_20.csv')
df_un_wiki1_50 = pd.read_csv('df_un_wiki1_50.csv')

time: 94 ms (started: 2021-05-07 13:45:55 +00:00)


In [23]:
print ("BALANCED")
print ("Average value for wiki1_1 "+ str (df_bal_wiki1_1['f1_score'].mean()))
print ("Average value for wiki1_2 "+ str (df_bal_wiki1_2['f1_score'].mean()))
print ("Average value for wiki1_3 "+ str (df_bal_wiki1_3['f1_score'].mean()))
print ("Average value for wiki1_5 "+ str (df_bal_wiki1_5['f1_score'].mean()))
print ("Average value for wiki1_7 "+ str (df_bal_wiki1_7['f1_score'].mean()))
print ("Average value for wiki1_10 "+ str (df_bal_wiki1_10['f1_score'].mean()))
print ("Average value for wiki1_20 "+ str (df_bal_wiki1_20['f1_score'].mean()))
print ("Average value for wiki1_50 "+ str (df_bal_wiki1_50['f1_score'].mean()))
print("UNATTESTED")
print ("Average value for wiki1_1 "+ str (df_un_wiki1_1['f1_score'].mean()))
print ("Average value for wiki1_2 "+ str (df_un_wiki1_2['f1_score'].mean()))
print ("Average value for wiki1_3 "+ str (df_un_wiki1_3['f1_score'].mean()))
print ("Average value for wiki1_5 "+ str (df_un_wiki1_5['f1_score'].mean()))
print ("Average value for wiki1_7 "+ str (df_un_wiki1_7['f1_score'].mean()))
print ("Average value for wiki1_10 "+ str (df_un_wiki1_10['f1_score'].mean()))
print ("Average value for wiki1_20 "+ str (df_un_wiki1_20['f1_score'].mean()))
print ("Average value for wiki1_50 "+ str (df_un_wiki1_50['f1_score'].mean()))

BALANCED
Average value for wiki1_1 0.7505199999999995
Average value for wiki1_2 0.7541199999999993
Average value for wiki1_3 0.7527299999999986
Average value for wiki1_5 0.745120000000001
Average value for wiki1_7 0.7413000000000022
Average value for wiki1_10 0.738310000000004
Average value for wiki1_20 0.7264000000000049
Average value for wiki1_50 0.7073900000000065
UNATTESTED
Average value for wiki1_1 0.5872999999999977
Average value for wiki1_2 0.5944000000000029
Average value for wiki1_3 0.5979800000000068
Average value for wiki1_5 0.5979800000000066
Average value for wiki1_7 0.5937000000000029
Average value for wiki1_10 0.5939500000000025
Average value for wiki1_20 0.5925400000000012
Average value for wiki1_50 0.5937700000000024
time: 29.1 ms (started: 2021-05-07 13:45:58 +00:00)
